# EEG Data Curation Part 1

2020-10-20 에 전달받은 EEG 데이터인 EDF 파일들과 `DB_list.xlsx` 메타데이터 파일을 살펴보고, 이후 학습을 위해 정리하는 노트북.

-----

## 파일 수작업 정리 1

이 노트북을 작성하는 동안, 일부 중복된 파일이나 메타데이터들을 수작업으로 통합하고 제거하였다.  
수작업 내용은 아래를 참고하자.

1. One-to-many matching case

   전달받은 `DB_list.xlsx`에 중복된 메타데이터 (병록/나이/진단) 행이 존재하는 경우이다.  
   이 경우, 중복된 정보들을 하나의 행으로 통합하였다.  
   구체적인 작업 방법은 `210316_중복제거_one-to-many.xlsx` 파일에 기록해두었다.

2. Many-to-one matching case

   동일한 병록번호를 가진 환자가 여러차례 EEG를 측정한 경우이다.  
   이 경우, 파일명과 용량을 토대로 중복 저장된 파일들을 제거해주었다.  
   또한 일부 파일들은 파일명을 변경해주었다.  
   구체적인 작업 방법은 `210504_many-to-one_Diagnosis_추가수정.xlsx` 파일에 기록해두었다.

3. Many-to-many matching case

   위에 언급된 두 가지 상황이 동시에 발생한 경우이다.  
   이 경우, (1) `DB_list.xlsx`에서 중복된 정보들을 하나로 통합하고, (2) 중복 저장된 파일들을 직접 제거하였다.  
   구체적인 작업 방법은 `210316_중복제거_many-to-many.xlsx` 파일에 기록해두었다.
   
업데이트된 메타데이터 파일은 `210330_DB_list_modified.xlsx`으로 저장해두었다.

-----

## 환경 구성

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Load some packages
import os
import re
import copy
import glob
import numpy as np
from openpyxl import load_workbook, Workbook, styles
import pprint
import datetime

# custom package
from utils.eeg_dataset import MultiLabel

In [ ]:
# Other settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # cleaner text

# plt.style.use('default') 
# ['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 
#  'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 
#  'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 
#  'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 
#  'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']

# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams["font.family"] = 'NanumGothic' # for Hangul in Windows

-----

## 데이터 파일 `(EDF, XLSX)` 구성 확인

In [ ]:
# Data file path
root_path = r'H:\Lab Database\2020 EEG\201020'
original_folder = r'01_Original_Data'

In [ ]:
count = 0
file_dict = {}
r = re.compile('[\\\/]([^\\\/]+)\.([^\\\/]+)$')

for i, f in enumerate(glob.glob(os.path.join(root_path, original_folder, '*.*'))):
    match_result = r.search(f)
    
    if match_result is None:
        print('Cannot parse this file:', f)
    
    fname = match_result.group(1)
    ext = match_result.group(2).lower()
    
    if ext not in file_dict.keys():
        file_dict.setdefault(ext, [])

    file_dict[ext].append(fname)
    count += 1

print('There are {:,} files in total, and their extensions are among of {}.'.format(count, list(file_dict.keys())))
for k, v in file_dict.items():
    print(' - {ext} {num:,} files'.format(ext=k, num=len(v)))

In [ ]:
unpaired_file_dict = copy.deepcopy(file_dict)
edf_paired = []

for fname in file_dict['edf']:
    if fname in file_dict['xlsx']:
        edf_paired.append(fname)
        unpaired_file_dict['edf'].remove(fname)
        unpaired_file_dict['xlsx'].remove(fname)

print('{:,} EDF and XLSX files are paired in total.'.format(len(edf_paired)))
print()
print('{:,} EDF and {:,} XLSX files are unmatched, respectively, and they are:'.format(len(unpaired_file_dict['edf']), len(unpaired_file_dict['xlsx'])))
pprint.pp(unpaired_file_dict)

del file_dict, unpaired_file_dict

-----

## 메타데이터 구성 확인

In [ ]:
# meta_file = os.path.join(root_path, original_folder, r'DB_list.xlsx')
meta_file = os.path.join(root_path, r'00_Data_Information/210330_DB_list_modified.xlsx')
ws = load_workbook(meta_file, data_only=True)['Sheet1']

meta_names = []
meta_dx1 = []
meta_birth = []

dx1_counter = {}
name_duplication_counter = {}

num = 2
while True:
    # (deprecated) field 1: age
    # a = ws.cell(row=num, column=1).value
    # a = a.strip(' \n') if a is not None else None
    
    # field 2: hosp_id
    n = ws.cell(row=num, column=2).value
    n = '{:>08s}'.format(n.strip(' \n')) if n is not None else None
    
    # field 3: dx_1
    d = ws.cell(row=num, column=3).value
    d = d.lower().strip(' \n') if d is not None else 'unknown'
    
    # (new) field 4: birth
    b = ws.cell(row=num, column=4).value

    # check whether the row is empty (which is EOF condition)
    if n is None:
        break

    # count the number of the name duplication
    if n in meta_names:
        name_duplication_counter[n] = name_duplication_counter.get(n, 1) + 1
    
    # count the total number of the emergence for each dx_1's value
    dx1_counter[d] = dx1_counter.get(d, 0) + 1

    # update information
    meta_names.append(n)
    meta_dx1.append(d)
    meta_birth.append(b)
    
    # move the pivot row
    num += 1

print('{:,} metadata is loaded.'.format(len(meta_names)))
print()

print('Among them,')

print(' - {:,} samples have the duplication of the name field. They are:'.format(sum([v for v in name_duplication_counter.values()])))
pprint.pp(name_duplication_counter)

temp = len(list(filter(lambda x: float(x) > 0 if x is not None else False, meta_birth)))
print(' - we know their birth of {:,} samples, but not of the other {:,} samples.'.format(temp, len(meta_birth) - temp))
print(' - there are {:,} types of diagnoses. Each is:'.format(len(dx1_counter)))
pprint.pp(dx1_counter)
print()

del temp, dx1_counter, name_duplication_counter

-----

## `(EDF, XLSX)` 데이터 $\longleftrightarrow$ 메타데이터 연결

In [ ]:
data = []

for f in edf_paired:
    # parse the first fragment of the file name
    parsed = re.match('^([0-9]+)_?', f).group(1)
    
    # When the same patient ID is already registered, 
    # add only the EDF file on the existing link
    registered = False
    for d in data:
        if len(d[1]) > 0 and d[1][0][0] == parsed:
            registered = True
            d[0].append(f)
            break
            
    # add the patient ID, and link between the EDF file and metadata
    if not registered:
        meta_temp = [m for m in zip(meta_names, meta_birth, meta_dx1) if m[0] == parsed]
        data.append(([f], meta_temp))

#  When there is no linked file for some metadata, make an empty connection
for n in meta_names:
    if not any([d[1][0][0] == n for d in data if len(d[1]) > 0]):
        meta_temp = [m for m in zip(meta_names, meta_birth, meta_dx1) if m[0] == n]
        data.append(([], meta_temp))
        
# Sanity check
if sum([len(d[0]) for d in data]) != len(edf_paired):
    print('ERROR 1')

if sum([len(d[1]) for d in data]) != len(meta_names):
    print('ERROR 2')


print('{:<30}: {:,}'.format('Total (EDF, XLSX) pair', len(edf_paired)))
print('{:<30}: {:,}'.format('Total rows of metadata', len(meta_names)))
print('-' * 70)

# print('{:<30}: {:,}'.format('Total data', len(data)))
# print()

print('- {:>5,} EDF data has no linked metadata.'.format(sum([len(d[1]) == 0 for d in data])))
# print([d[0][0] for d in data if len(d[1]) == 0])
# print('-' * 70)

print('- {:>5,} metadata has no linked EDf file.'.format(sum([len(d[0]) == 0 for d in data])))
# print([d[1][0][0] for d in data if len(d[0]) == 0])
# print('-' * 70)

data = [d for d in data if len(d[1]) > 0 and len(d[0]) > 0]
print('= Only {:>5,} data has the connection between them.'.format(len(data)))
print('-' * 70)

print('Among them,')
print(f'- {sum([len(d[0]) == 1 and len(d[1]) == 1 for d in data]):>5,} are the   one-to-one matching between (EDF - metadata).')
print(f'- {sum([len(d[0]) > 1 and len(d[1]) == 1 for d in data]):>5,} are the  many-to-one matching between (EDF - metadata).')
print(f'- {sum([len(d[0]) == 1 and len(d[1]) > 1 for d in data]):>5,} are the  one-to-many matching between (EDF - metadata).')
print(f'- {sum([len(d[0]) > 1 and len(d[1]) > 1 for d in data]):>5,} are the many-to-many matching between (EDF - metadata).')

del edf_paired, meta_names, meta_birth, meta_dx1

-----

## `(EDF, XLSX)` 데이터 $\longleftrightarrow$ 메타데이터 톺아보기

#### 데이터 톺아보기 1

- 다대일, 일대다, 다대다 매칭 목록 작성 ($\Rightarrow$ 손수 데이터를 정제하는데 참고)

In [ ]:
# list-up one-to-many matching in excel sheet
wb = Workbook()
ws = wb.active
ws.title = 'one-to-many'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'Metadata'
ws.cell(row=1, column=3).value = 'birth'
ws.cell(row=1, column=4).value = 'dx_1'
ws.cell(row=1, column=5).value = 'New birth'
ws.cell(row=1, column=6).value = 'New dx_1'

(r_pivot, r_max, c_counter) = (2, 2, 0)
for d in data:
    if len(d[0]) == 1 and len(d[1]) > 1:
        # edf
        for (k, edf_name) in enumerate(d[0]):
            r_max = max(r_max, r_pivot + k)
            ws.cell(row=r_pivot + k, column=1).value = edf_name
        # metadata
        for (k, meta) in enumerate(d[1]):
            r_max = max(r_max, r_pivot + k)
            ws.cell(row=r_pivot + k, column=2).value = meta[0]
            ws.cell(row=r_pivot + k, column=3).value = meta[1]
            ws.cell(row=r_pivot + k, column=4).value = meta[2]
        # coloring
        for rows in ws.iter_rows(min_row=r_pivot, max_row=r_max, min_col=1, max_col=6):
            for cell in rows:
                color = 'FFFFCC' if c_counter % 2 == 0 else '00C0C0C0'
                cell.fill = styles.PatternFill(start_color=color, end_color=color, fill_type="solid")
        r_pivot = r_max + 1
        c_counter += 1

wb.save('output/one-to-many.xlsx')
print('one-to-many.xlsx is saved.')

# list-up many-to-one matching in excel sheet
wb = Workbook()
ws = wb.active
ws.title = 'many-to-one'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'Metadata name'
ws.cell(row=1, column=3).value = 'Metadata birth'
ws.cell(row=1, column=4).value = 'dx_1'
ws.cell(row=1, column=5).value = 'Delete?'

(r_pivot, r_max, c_counter) = (2, 2, 0)
for d in data:
    if len(d[0]) > 1 and len(d[1]) == 1:
        # edf
        for (k, edf_name) in enumerate(d[0]):
            r_max = max(r_max, r_pivot + k)
            ws.cell(row=r_pivot + k, column=1).value = edf_name
        # metadata
        for (k, meta) in enumerate(d[1]):
            r_max = max(r_max, r_pivot + k)
            ws.cell(row=r_pivot + k, column=2).value = meta[0]
            ws.cell(row=r_pivot + k, column=3).value = meta[1]
            ws.cell(row=r_pivot + k, column=4).value = meta[2]
        # coloring
        for rows in ws.iter_rows(min_row=r_pivot, max_row=r_max, min_col=1, max_col=5):
            for cell in rows:
                color = 'FFFFCC' if c_counter % 2 == 0 else '00C0C0C0'
                cell.fill = styles.PatternFill(start_color=color, end_color=color, fill_type="solid")
        r_pivot = r_max + 1
        c_counter += 1
        
wb.save('output/many-to-one.xlsx')
print('many-to-one.xlsx is saved.')

# list-up many-to-many matching in excel sheet
wb = Workbook()
ws = wb.active
ws.title = 'many-to-many'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'Metadata name'
ws.cell(row=1, column=3).value = 'Metadata birth'
ws.cell(row=1, column=4).value = 'dx_1'
ws.cell(row=1, column=5).value = 'New birth'
ws.cell(row=1, column=6).value = 'New dx_1'

(r_pivot, r_max, c_counter) = (2, 2, 0)
for d in data:
    if len(d[0]) > 1 and len(d[1]) > 1:
        # edf
        for (k, edf_name) in enumerate(d[0]):
            r_max = max(r_max, r_pivot + k)
            ws.cell(row=r_pivot + k, column=1).value = edf_name
        # metadata
        for (k, meta) in enumerate(d[1]):
            r_max = max(r_max, r_pivot + k)
            ws.cell(row=r_pivot + k, column=2).value = meta[0]
            ws.cell(row=r_pivot + k, column=3).value = meta[1]
            ws.cell(row=r_pivot + k, column=4).value = meta[2]
        # coloring
        for rows in ws.iter_rows(min_row=r_pivot, max_row=r_max, min_col=1, max_col=6):
            for cell in rows:
                color = 'FFFFCC' if c_counter % 2 == 0 else 'C0C0C0'
                cell.fill = styles.PatternFill(start_color=color, end_color=color, fill_type="solid")
        r_pivot = r_max + 1
        c_counter += 1
        
wb.save('output/many-to-many.xlsx')
print('many-to-many.xlsx is saved.')

#### 데이터 톺아보기 2

- 일대일 대응만 추려내기
- 그 중 (Dementia, MCI, Normal) 진단명을 갖는 데이터 수 통계

In [ ]:
# pull out some data
# 1. one-to-one matching between (EDF, XLSX) and (metadata)
# 2. with the diagnoses among (normal, mci, dementia)
# (regardless of birth value)
one_to_one_data = [(d[0][0], d[1][0][1], MultiLabel.load_from_string(d[1][0][2])) for d in data 
                   if len(d[0]) == 1 and len(d[1]) == 1]

# count the numbers of patients in the category of 'normal,' 'mci,' 'dementia.'
(num_normal, num_mci, num_dementia, num_others) = (0, 0, 0, 0)
(num_ef, num_rf) = (0, 0)
for d in one_to_one_data:
    if d[2].check('dementia'):
        num_dementia += 1
    elif d[2].check('mci'):
        num_mci += 1
        if d[2].check('mci_amnestic_ef'):
            num_ef += 1
        elif d[2].check('mci_amnestic_rf'):
            num_rf += 1
    elif d[2].check('normal'):
        num_normal += 1
    else:
        num_others += 1

print(f'The number of one-to-one data is \t\t\t{len(one_to_one_data):>5,}')
print('Among them,')
print(f' - # of data with <Dementia> diagnosis is \t\t{num_dementia:>5,}')
print(f'   * # of data with <MCI> diagnosis is \t\t\t{num_mci:>5,}')
print(f'   * # of data with <MCI-Amnestic-EF> diagnosis is \t\t{num_ef:>5,}')
print(f' - # of data with <MCI-Amnestic-RF> diagnosis is \t\t{num_rf:>5,}')
print(f' - # of data with <Normal> diagnosis is \t\t{num_normal:>5,}')
print(f' - # of data with the other diagnoses is \t\t{num_others:>5,}')
print()

# save the filtered data
wb = Workbook()
ws = wb.active
ws.title = 'one_to_one_data'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'birth'
for (i, label) in enumerate(one_to_one_data[0][2].get_label_types()):
    ws.cell(row=1, column=3 + i).value = label

for (i, d) in enumerate(one_to_one_data):
    ws.cell(row=2 + i, column=1).value = d[0] 
    ws.cell(row=2 + i, column=2).value = d[1] if d[1] is not None and float(d[1]) > 0 else None
    for (k, label) in enumerate(d[2].get_label_values()):
        ws.cell(row=2 + i, column=3 + k).value = label if label is not False else None

    # coloring
    color = 'FDFDD0' if i % 2 == 0 else 'D9E5FF'
    for rows in ws.iter_rows(min_row=2 + i, max_row=2 + i, min_col=1, max_col=2 + d[2].get_size()):
        for cell in rows:
            cell.fill = styles.PatternFill(start_color=color, end_color=color, fill_type="solid")

wb.save('output/filtered_one_to_one_data.xlsx')
print('filtered_one_to_one_data.xlsx is saved.')

del one_to_one_data

#### 데이터 톺아보기 3

- 일대일 대응
- 유효한 나이 값
- 그 중 (Dementia, MCI, Normal) 진단명을 갖는 데이터 수 통계

In [ ]:
# pull out some data
# 1. one-to-one matching between (EDF, XLSX) and (metadata)
# 2. with the valid birth value
# 3. with the diagnoses among (normal, mci, dementia)
one_to_one_aged_data = [(d[0][0], d[1][0][1], MultiLabel.load_from_string(d[1][0][2])) for d in data 
                        if len(d[0]) == 1 and len(d[1]) == 1 and 
                        d[1][0][1] is not None and float(d[1][0][1]) > 0]

# count the numbers of patients in the category of 'normal,' 'mci,' 'dementia.'
(num_normal, num_mci, num_dementia, num_others) = (0, 0, 0, 0)
(num_ef, num_rf) = (0, 0)
for d in one_to_one_aged_data:
    if d[2].check('dementia'):
        num_dementia += 1
    elif d[2].check('mci'):
        num_mci += 1
        if d[2].check('mci_amnestic_ef'):
            num_ef += 1
        elif d[2].check('mci_amnestic_rf'):
            num_rf += 1
    elif d[2].check('normal'):
        num_normal += 1
    else:
        num_others += 1

print(f'The number of one-to-one aged data is \t\t\t{len(one_to_one_aged_data):>5,}')
print('Among them,')
print(f' - # of data with <Dementia> diagnosis is \t\t{num_dementia:>5,}')
print(f'   * # of data with <MCI> diagnosis is \t\t\t{num_mci:>5,}')
print(f'   * # of data with <MCI-Amnestic-EF> diagnosis is \t\t{num_ef:>5,}')
print(f' - # of data with <MCI-Amnestic-RF> diagnosis is \t\t{num_rf:>5,}')
print(f' - # of data with <Normal> diagnosis is \t\t{num_normal:>5,}')
print(f' - # of data with the other diagnoses is \t\t{num_others:>5,}')
print()
      
# save the filtered data
wb = Workbook()
ws = wb.active
ws.title = 'one_to_one_aged_data'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'birth'
for (i, label) in enumerate(one_to_one_aged_data[0][2].get_label_types()):
    ws.cell(row=1, column=3 + i).value = label

for (i, d) in enumerate(one_to_one_aged_data):
    ws.cell(row=2 + i, column=1).value = d[0]
    ws.cell(row=2 + i, column=2).value = d[1]
    for (k, label) in enumerate(d[2].get_label_values()):
        ws.cell(row=2 + i, column=3 + k).value = label if label is not False else None

    # coloring
    color = 'FDFDD0' if i % 2 == 0 else 'D9E5FF'
    for rows in ws.iter_rows(min_row=2 + i, max_row=2 + i, min_col=1, max_col=2 + d[2].get_size()):
        for cell in rows:
            cell.fill = styles.PatternFill(start_color=color, end_color=color, fill_type="solid")
        
wb.save('output/filtered_one_to_one_aged_data.xlsx')
print('filtered_one_to_one_aged_data.xlsx is saved.')

del one_to_one_aged_data

#### 데이터 톺아보기 4

- 유효하지 않은 나이 값 ($\Rightarrow$ 나이 값 업데이트 필요한 데이터가 무엇인지 파악)

In [ ]:
# pull out some data
# 1. with *INVALID* birth values

data_temp = []
for d in data:
    for edf in d[0]:
        data_temp.append((edf, d[1]))

invalid_age_data = [(d[0], d[1][0][1], d[1][0][2]) for d in data_temp 
                   if d[1][0][1] is None or float(d[1][0][1]) < 0]
invalid_age_data = sorted(invalid_age_data, key=lambda x: x[0])

print(f'The number of one-to-one data is {len(invalid_age_data):>5,}')

# save the filtered data
wb = Workbook()
ws = wb.active
ws.title = 'invalid_age_data'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'birth'
ws.cell(row=1, column=3).value = 'dx_1'

for (i, d) in enumerate(invalid_age_data):
    ws.cell(row=2 + i, column=1).value = d[0]
    ws.cell(row=2 + i, column=2).value = d[1]
    ws.cell(row=2 + i, column=3).value = d[2]
    
wb.save('output/filtered_invalid_age_data.xlsx')
print('filtered_invalid_age_data.xlsx is saved.')

del invalid_age_data

#### 데이터 톺아보기 5

- 다대일 대응, 또는 다대다 대응
- 측정 시간차가 큰 순으로 정렬 ($\Rightarrow$ 진단명 업데이트가 필요한 데이터가 무엇인지 파악)

In [ ]:
# pull out some data
# 1. many-to-one or many-to-many matching between (EDF, XLSX) and (metadata)
# 2. sort the data by the time delta between EEG measurements

many_edf_data =[[(edf_name, datetime.date(2000 + int(edf_name[-2:]), int(edf_name[-4:-2]), int(edf_name[-6:-4])) 
                  if len(edf_name) == 15 else datetime.date.max) for edf_name in d[0]] for d in data if len(d[0]) > 1]

many_edf_data_temp = []
for d in many_edf_data:
    d = sorted(d, key=lambda x: x[1])
    many_edf_data_temp.append(d)

many_edf_data = sorted(many_edf_data_temp, key=lambda x: x[-1][1] - x[0][1], reverse=True)
del many_edf_data_temp

# list-up many edf data matching in excel sheet
wb = Workbook()
ws = wb.active
ws.title = 'many_edf_data'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'Time delta (years)'
ws.cell(row=1, column=3).value = '파일명이 잘못 되었을 경우, 새로운 파일명'

(r_pivot, r_max, c_counter) = (2, 2, 0)
for d in many_edf_data:
    # edf
    for (k, edf_name) in enumerate(d):
        r_max = max(r_max, r_pivot + k)
        ws.cell(row=r_pivot + k, column=1).value = edf_name[0]
        
    # time delta
    ws.cell(row=r_pivot, column=2).value = '%.2f' % ((d[-1][1] - d[0][1]).days / 365)

    # coloring
    for rows in ws.iter_rows(min_row=r_pivot, max_row=r_max, min_col=1, max_col=3):
        for cell in rows:
            color = 'FAF4C0' if c_counter % 2 == 0 else 'B2CCFF'
            cell.fill = styles.PatternFill(start_color=color, end_color=color, fill_type="solid")
    r_pivot = r_max + 1
    c_counter += 1
        
wb.save('output/many_edf_data.xlsx')
print('many_edf_data.xlsx is saved.')

del many_edf_data

#### 데이터 톺아보기 6

- 일대일 대응, 그에 더해
- 다대일 대응 데이터를 쪼개서 여러 개의 일대일 대응 데이터로 변환

In [ ]:
# pull out some data
# 1. one-to-one matching between (EDF, XLSX) and (metadata)
# 2. split many-to-one matching into many one-to-one matching

splitted_many_data = []
for d in data:
    # one edf vs one metadata case
    if len(d[0]) == 1 and len(d[1]) == 1:
        birth = d[1][0][1] if d[1][0][1] is not None and float(d[1][0][1]) > 0 else None
        dx1 = d[1][0][2]
        splitted_many_data.append((d[0][0], birth, dx1, MultiLabel.load_from_string(d[1][0][2])))
    # many edfs vs one metadata case
    elif len(d[0]) >= 1 and len(d[1]) == 1:
        birth = d[1][0][1] if d[1][0][1] is not None and float(d[1][0][1]) > 0 else None
        dx1 = d[1][0][2]
        splitted_many_data.extend([(edf_name, birth, dx1, MultiLabel.load_from_string(d[1][0][2])) for edf_name in d[0]])
    # one edf vs many metadata case
    elif len(d[0]) == 1 and len(d[1]) >= 1:
        pass
    # many edfs vs many metadata case
    else:
        pass

print(f'After splitted, the number of one-to-one data is \t{len(splitted_many_data):>5,}')

# count the numbers of patients in the category of 'normal,' 'mci,' 'dementia.'
(num_normal, num_mci, num_dementia, num_others) = (0, 0, 0, 0)
(num_ef, num_rf) = (0, 0)
for d in splitted_many_data:
    if d[3].check('dementia'):
        num_dementia += 1
    elif d[3].check('mci'):
        num_mci += 1
        if d[3].check('mci_amnestic_ef'):
            num_ef += 1
        elif d[3].check('mci_amnestic_rf'):
            num_rf += 1
    elif d[3].check('normal'):
        num_normal += 1
    else:
        num_others += 1

print('Among them,')
print(f' - # of data with <Dementia> diagnosis is \t\t{num_dementia:>5,}')
print(f'   - # of data with <MCI> diagnosis is \t\t\t{num_mci:>5,}')
print(f'   - # of data with <MCI-Amnestic-EF> diagnosis is \t\t{num_ef:>5,}')
print(f' - # of data with <MCI-Amnestic-RF> diagnosis is \t\t{num_rf:>5,}')
print(f' - # of data with <Normal> diagnosis is \t\t{num_normal:>5,}')
print(f' - # of data with the other diagnoses is \t\t{num_others:>5,}')
print()

# save the filtered data
wb = Workbook()
ws = wb.active
ws.title = 'splitted_many_data'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'birth'
ws.cell(row=1, column=3).value = 'dx1'
for (i, label) in enumerate(splitted_many_data[0][3].get_label_types()):
    ws.cell(row=1, column=4 + i).value = label

for (i, d) in enumerate(splitted_many_data):
    ws.cell(row=2 + i, column=1).value = d[0]
    ws.cell(row=2 + i, column=2).value = d[1]
    ws.cell(row=2 + i, column=3).value = d[2]
    for (k, label) in enumerate(d[3].get_label_values()):
        ws.cell(row=2 + i, column=4 + k).value = label if label is not False else None
    
    # coloring
    color = 'FDFDD0' if i % 2 == 0 else 'D9E5FF'
    for rows in ws.iter_rows(min_row=2 + i, max_row=2 + i, min_col=1, max_col=3 + d[3].get_size()):
        for cell in rows:
            cell.fill = styles.PatternFill(start_color=color, end_color=color, fill_type="solid")
    
wb.save('output/filtered_splitted_many_data.xlsx')
print('filtered_splitted_many_data.xlsx is saved.')

metadata_temp = splitted_many_data
del splitted_many_data

#### 데이터 톺아보기 7

- 일대일 대응, 그에 더해
- 다대일 대응 데이터 중 하나만 선택 (측정일시 기준으로 가장 마지막 데이터만)

In [ ]:
# pull out some data
# 1. one-to-one matching between (EDF, XLSX) and (metadata)
# 2. select the most recently recorded EDF among many-to-one matching

selected_many_data = []
for d in data:
    # one edf vs one metadata case
    if len(d[0]) == 1 and len(d[1]) == 1:
        birth = d[1][0][1] if d[1][0][1] is not None and float(d[1][0][1]) > 0 else None
        selected_many_data.append((d[0][0], birth, MultiLabel.load_from_string(d[1][0][2])))
    # many edfs vs one metadata case
    elif len(d[0]) >= 1 and len(d[1]) == 1:
        (last, idx) = (datetime.date.min, 0)
        for (k, edf_name) in enumerate(d[0]):
            if len(edf_name) != 15: 
                continue
            date = datetime.date(2000 + int(edf_name[-2:]), int(edf_name[-4:-2]), int(edf_name[-6:-4]))
            if last < date:
                (last, idx) = (date, k)
        birth = d[1][0][1] if d[1][0][1] is not None and float(d[1][0][1]) > 0 else None
        selected_many_data.append((d[0][idx], birth, MultiLabel.load_from_string(d[1][0][2]))) 
    # one edf vs many metadata case
    elif len(d[0]) == 1 and len(d[1]) >= 1:
        pass
    # many edfs vs many metadata case
    else:
        pass

print(f'After splitted, the number of one-to-one data is \t{len(selected_many_data):>5,}')

# count the numbers of patients in the category of 'normal,' 'mci,' 'dementia.'
(num_normal, num_mci, num_dementia, num_others) = (0, 0, 0, 0)
(num_ef, num_rf) = (0, 0)
for d in selected_many_data:
    if d[2].check('dementia'):
        num_dementia += 1
    elif d[2].check('mci'):
        num_mci += 1
        if d[2].check('mci_amnestic_ef'):
            num_ef += 1
        elif d[2].check('mci_amnestic_rf'):
            num_rf += 1
    elif d[2].check('normal'):
        num_normal += 1
    else:
        num_others += 1

print('Among them,')
print(f' - # of data with <Dementia> diagnosis is \t\t{num_dementia:>5,}')
print(f'   - # of data with <MCI> diagnosis is \t\t\t{num_mci:>5,}')
print(f'   - # of data with <MCI-Amnestic-EF> diagnosis is \t\t{num_ef:>5,}')
print(f' - # of data with <MCI-Amnestic-RF> diagnosis is \t\t{num_rf:>5,}')
print(f' - # of data with <Normal> diagnosis is \t\t{num_normal:>5,}')
print(f' - # of data with the other diagnoses is \t\t{num_others:>5,}')
print()

# save the filtered data
wb = Workbook()
ws = wb.active
ws.title = 'selected_many_data'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'birth'
for (i, label) in enumerate(selected_many_data[0][2].get_label_types()):
    ws.cell(row=1, column=3 + i).value = label

for (i, d) in enumerate(selected_many_data):
    ws.cell(row=2 + i, column=1).value = d[0]
    ws.cell(row=2 + i, column=2).value = d[1]
    for (k, label) in enumerate(d[2].get_label_values()):
        ws.cell(row=2 + i, column=3 + k).value = label if label is not False else None
    
    # coloring
    color = 'FDFDD0' if i % 2 == 0 else 'D9E5FF'
    for rows in ws.iter_rows(min_row=2 + i, max_row=2 + i, min_col=1, max_col=2 + d[2].get_size()):
        for cell in rows:
            cell.fill = styles.PatternFill(start_color=color, end_color=color, fill_type="solid")
    
wb.save('output/filtered_selected_many_data.xlsx')
print('filtered_selected_many_data.xlsx is saved.')

del selected_many_data

-----

## 메타데이터 정리하여 새로 저장하기

In [ ]:
# Save the metadata
# 1. one-to-one matching between (EDF, XLSX) and (metadata)
# 2. split many-to-one matching into many one-to-one matching

new_metadata = []
for d in data:
    # one edf vs one metadata case
    if len(d[0]) == 1 and len(d[1]) == 1:
        birth = d[1][0][1]
        dx1 = d[1][0][2]
        new_metadata.append((d[0][0], birth, dx1))
    # many edfs vs one metadata case
    elif len(d[0]) >= 1 and len(d[1]) == 1:
        birth = d[1][0][1]
        dx1 = d[1][0][2]
        new_metadata.extend([(edf_name, birth, dx1) for edf_name in d[0]])
    # one edf vs many metadata case
    elif len(d[0]) == 1 and len(d[1]) >= 1:
        pass
    # many edfs vs many metadata case
    else:
        pass

# save new metadata as XLSX
wb = Workbook()
ws = wb.active
ws.title = 'metadata'
ws.cell(row=1, column=1).value = 'EDF filename'
ws.cell(row=1, column=2).value = 'dx1'
ws.cell(row=1, column=3).value = 'birth'

for (i, d) in enumerate(new_metadata):
    ws.cell(row=2 + i, column=1).value = d[0]
    ws.cell(row=2 + i, column=2).value = d[2] # dx1
    ws.cell(row=2 + i, column=3).value = d[1] # birth
    
    # coloring
    color = 'FDFDD0' if i % 2 == 0 else 'D9E5FF'
    for rows in ws.iter_rows(min_row=2 + i, max_row=2 + i, min_col=1, max_col=3):
        for cell in rows:
            cell.fill = styles.PatternFill(start_color=color, end_color=color, fill_type="solid")
    
wb.save('output/DB_list.xlsx')
print('DB_list.xlsx is saved.')

metadata_temp = new_metadata
del new_metadata